In [2]:
import quartz
from quartz.core import PyGraph

ImportError: libquartz_runtime.so: cannot open shared object file: No such file or directory

## Construct QuartzContext
The following lines are used to construct a context in quartz which contains all the generated transfers under a given gate set.

In [ ]:
# Construct a context
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')

In [ ]:
quartz_context.num_xfers

In [ ]:
quartz_context.get_xfers()

In [ ]:
quartz_context.get_xfer_from_id(id=0)

## Construct PyGraph

In [ ]:
# Construct a PyGraph from qasm file
my_graph = PyGraph.from_qasm(context=quartz_context, filename="barenco_tof_3_opt_path/subst_history_39.qasm")

In [ ]:
# Construct a PyGraph from qasm string
my_new_graph = PyGraph.from_qasm_str(context=quartz_context, qasm_str=my_graph.to_qasm_str())

## PyGraph

Below are some examples showing the APIs in PyGraph.

In [ ]:
my_graph.num_nodes, my_graph.num_edges

In [ ]:
my_graph.all_edges()

In [ ]:
my_graph.all_nodes()

In [ ]:
my_graph.all_nodes_with_id()

In [ ]:
my_graph.get_node_from_id(id=0)

In [ ]:
my_graph.to_qasm(filename="test.qasm")

In [ ]:
my_graph_dgl = my_graph.to_dgl_graph()

In [ ]:
my_graph_dgl.num_edges()

In [ ]:
my_graph_dgl.edata

In [ ]:
my_graph_dgl.ndata

In [ ]:
my_graph_dgl.num_edges()

In [ ]:
all_nodes = my_graph.all_nodes()
all_nodes

In [ ]:
my_graph.depth

The codes below shows the usage of the two APIs `PyGraph.available_xfers` and `PyGraph.apply_xfer`.

In [ ]:
# available_xfer_matrix = my_graph.get_available_xfers_matrix(context=quartz_context)
# for node in all_nodes:
#     print(my_graph.available_xfers(context=quartz_context, node=node))

In [ ]:
new_graph = my_graph.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=3259), node=all_nodes[0])

In [ ]:
new_graph.hash()

In [ ]:
new_graph.gate_count

The codes below is a back tracking search implemented with the quartz python APIs.

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

def check(graph):
    graph.to_qasm(filename='best.qasm')
    qc_origin = QuantumCircuit.from_qasm_file('barenco_tof_3_opt_path/subst_history_39.qasm')
    qc_optimized = QuantumCircuit.from_qasm_file('best.qasm')
    return Statevector.from_instruction(qc_origin).equiv(Statevector.from_instruction(qc_optimized))

In [ ]:
# Optimizing with BFS
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy

candidate_hq = []
# heapq.heappush(candidate_hq, my_graph)
heapq.heappush(candidate_hq, (my_graph, []))
hash_set = set()
hash_set.add(my_graph.hash())
best_graph = my_graph
best_graph_trace = []
best_gate_cnt = my_graph.gate_count
q_max_len = 1000

budget = 1_000_000

# while candidate_hq != [] and budget >= 0:
#     first_candidate = heapq.heappop(candidate_hq)
#     print(first_candidate.gate_count)
#     try:
#         all_nodes = first_candidate.all_nodes()
#     except:
#         print(first_candidate)
#         print(first_candidate.gate_count)
#         print(first_candidate.num_nodes)
#         exit(1)
#     for node in all_nodes:
#         appliable_xfers = first_candidate.available_xfers(context=quartz_context, node=node)
#         for xfer in appliable_xfers:
#             new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
#             if new_graph.hash() not in hash_set:
#                 hash_set.add(new_graph.hash())
#                 heapq.heappush(candidate_hq, new_graph)
#                 if new_graph < best_graph:
#                     best_graph = new_graph
#                     best_gate_cnt = new_graph.gate_count
#                 budget -= 1
#                 if budget % 1_000 == 0:
#                     print(f'{budget}: minimum gate count is {best_gate_cnt}')

while candidate_hq != [] and budget >= 0:
    first = heapq.heappop(candidate_hq)
    first_candidate = first[0]
    first_candidate_trace = first[1]
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=24) as executor:
        results = executor.map(ax, list(range(len(all_nodes))))
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            if new_graph.hash() not in hash_set:
                hash_set.add(new_graph.hash())
                new_graph_trace = copy.deepcopy(first_candidate_trace)
                new_graph_trace.append((i, xfer))
                heapq.heappush(candidate_hq, (new_graph, new_graph_trace))
                if len(candidate_hq) > q_max_len:
                    candidate_hq = candidate_hq[:-1]
                if new_graph < best_graph:
                    best_graph = new_graph
                    best_graph_trace = copy.deepcopy(new_graph_trace)
                    best_gate_cnt = new_graph.gate_count
                elif new_graph.gate_count == best_graph.gate_count:
                    if len(new_graph_trace) < len(best_graph_trace):
                        best_graph = new_graph
                        best_graph_trace = copy.deepcopy(new_graph_trace)
                if best_gate_cnt == 40:
                    break
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}, correctness: {check(best_graph)}')
                    print(f'Best graph trace: {best_graph_trace}')
